In [8]:
import pandas as pd
import numpy as np

block_meta = pd.read_csv('..//data//created//block_meta.csv')

data = pd.read_hdf('..//data//model//model_data_12H.h5')
data.drop('CaseID', axis = 1, inplace = True)

#data['has_poop'] = data['poop'].groupby(data['block_fips']).transform('sum')
#data = data[data['has_poop']>0]

block_neigh = pd.read_csv('..//data//created//block_neigh.csv')
block_neigh = block_neigh[block_neigh['nbrhood_name'].notnull()]
block_neigh = block_neigh.loc[:,[
    
    'block_fips',
    'nbrhood_name'
]]

In [9]:
data = data.merge(block_neigh, on = 'block_fips')

data['Opened_rnd'] = pd.to_datetime(data['Opened_rnd'], format="%Y-%m-%d %H:%M:%S")
data['day_of_week'] = data['Opened_rnd'].dt.dayofweek
data['month'] = data['Opened_rnd'].dt.month
data['hour'] = data['Opened_rnd'].dt.hour

data['winter'] = np.where(data['month'].isin([1, 2, 3]), 1, 0)
data['spring'] = np.where(data['month'].isin([4, 5, 6]), 1, 0)
data['summer'] = np.where(data['month'].isin([7, 8, 9]), 1, 0)
data['fall'] = np.where(data['month'].isin([10, 11, 12]), 1, 0)

In [10]:
neigh_dummies = pd.get_dummies(data['nbrhood_name'])
data = pd.concat((data,neigh_dummies), axis=1)
del neigh_dummies

month_dummies = pd.get_dummies(data['month'], prefix='month')
data = pd.concat((data,month_dummies), axis=1)
del month_dummies

hour_dummies = pd.get_dummies(data['hour'], prefix='hour')
data = pd.concat((data,hour_dummies), axis=1)
del hour_dummies

dow_dummies = pd.get_dummies(data['day_of_week'], prefix='dow')
data = pd.concat((data,dow_dummies), axis=1)
del dow_dummies

In [11]:
data = data.merge(block_meta, on='block_fips')

In [12]:
data[['nbrhood_name','poop']].groupby('nbrhood_name').mean().sort_values('poop', ascending = False)

,poop
nbrhood_name,
South of Market,0.015200
Inner Mission,0.013547
Cole Valley/Parnassus Heights,0.013274
Duboce Triangle,0.009385
Miraloma Park,0.009368
Downtown,0.007985
Eureka Valley / Dolores Heights,0.005640
Van Ness/Civic Center,0.005182
Hayes Valley,0.005007


In [13]:
neighs = [
    
    'Tenderloin',
    'South of Market',
    'Downtown',
    'Inner Mission',
    'Van Ness/Civic Center',
    'Duboce Triangle',
    'Haight Ashbury',
    'Miraloma Park',
    'Financial District/Barbary Coast',
    'Silver Terrace' 
]

data_small = data[data['nbrhood_name'].isin(neighs)]

In [14]:
data_small.to_hdf('..//data//model//data_to_model.h5', key='xyz', complib='blosc')

/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:1471: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block3_values] [items->['block_fips', 'nbrhood_name', 'HH_income', 'total_age', 'male_age', 'fem_age']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)
